In [23]:
import sys
import xarray as xr
import numpy as np
import pandas as pd
import math
import glob
import yaml
import geopandas as gpd
import shapely.geometry
import cartopy
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import matplotlib.ticker as mticker
import cmocean.cm as cmo

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature
import dask

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import roundPartial, select_months_ds
from plotter import draw_basemap, plot_terrain
from colorline import colorline

dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
server='skyriver'
if server == 'comet':
    path_to_data = '/data/projects/Comet/cwp140/'
elif server == 'expanse':
    path_to_data = '/expanse/lustre/scratch/dnash/temp_project/'
elif server == 'skyriver':
    path_to_data = '/home/dnash/comet_data/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [24]:
## Load Basin watershed file
fp = path_to_data + 'downloads/UpperCO/Upper_Colorado_River_Basin_Boundary/Upper_Colorado_River_Basin_Boundary.shp'
basin = gpd.read_file(fp, crs="EPSG:4326") # have to manually set the projection
basin

,EXT_ID,EXT_TYP_ID,NAME,geometry
0,9389,5.0,Upper Colorado Region,"POLYGON ((-106.65587 40.51559, -106.65376 40.5..."


In [3]:
## load cluster results
df_lst = []
for i in range(1, 5):
    fname = path_to_data + 'preprocessed/suma/Cluster{0}_PRISM_CO_Watershed_90th_percentile_precip_dates.csv'.format(i)
    df = pd.read_csv(fname)
    df = df.rename(columns={"# Cluster{0}_date".format(i): "date"})
    df['date'] = pd.to_datetime(df['date'])
    df['cluster'] = i
    df_lst.append(df)
tmp = pd.concat(df_lst, ignore_index=True)
tmp.set_index('date', inplace=True)
tmp

,cluster
date,
2000-01-27,1
2000-02-18,1
2000-03-16,1
2000-03-21,1
2000-03-31,1
...,...
2019-10-05,4
2019-11-21,4
2019-11-26,4


In [4]:
## load the list of HUC8s in Upper Colorado River Basin
fname = '../data/UpperColorado_HUC8s.txt'
HUC8_lst = pd.read_csv(fname, header=None, names=['idx', 'HUC8'], sep='\s+')
HUC8_lst = HUC8_lst.HUC8.values

In [5]:
ds_lst = []
for i, HUC8_ID in enumerate(HUC8_lst):
    fname = '/home/dnash/comet_data/preprocessed/ERA5_trajectories/final/PRISM_HUC8_{0}.nc'.format(HUC8_ID)
    ds = xr.open_dataset(fname)

    ## where start_date == cluster_date, assign cluster value
    cluster_lst = []
    start_dates = ds.start_date.values
    for j, date in enumerate(start_dates):
        idx = (tmp.index == date)
        cluster = tmp.loc[idx].cluster.values
        cluster_lst.append(cluster)
    result = np.concatenate(cluster_lst)

    ds = ds.assign({'cluster': ('start_date', result)})
    ds_lst.append(ds)

In [6]:
## concat ds_lst along HUC8 index
ds = xr.concat(ds_lst, pd.Index(HUC8_lst, name="HUC8"))
ds

<xarray.Dataset>
Dimensions:       (index: 72, start_date: 958, HUC8: 47)
Coordinates:
  * index         (index) int64 0 1 2 3 4 5 6 7 8 ... 63 64 65 66 67 68 69 70 71
  * start_date    (start_date) datetime64[ns] 2000-01-11 ... 2019-12-28
    lat           (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    lon           (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    time          (HUC8, start_date, index) datetime64[ns] NaT NaT ... NaT NaT
  * HUC8          (HUC8) int64 13020102 14080101 13010005 ... 10190006 10190007
Data variables: (12/14)
    IVT           (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    uIVT          (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    vIVT          (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    IWV           (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    level         (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    q             (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    ...            ...
    w             (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    dq            (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    drying_ratio  (HUC8, start_date, index) float64 nan nan nan ... nan nan nan
    ar_scale      (HUC8, start_date) float64 nan nan nan 3.0 ... nan nan nan nan
    ar            (HUC8, start_date) float64 nan nan nan 1.0 ... nan nan nan nan
    cluster       (HUC8, start_date) float64 nan nan nan 2.0 ... nan nan nan nan
Attributes:
    Conventions:                CF-1/0
    history:                    Fri Jul 17 10:26:21 2020: /apps/nco-4.9.3_gnu...
    history_of_appended_files:  Fri Jul 17 10:26:21 2020: Appended file ERA5_...
    NCO:                        netCDF Operators version 4.9.3 (Homepage = ht...

In [18]:
cell_lst = []
for i in range(1, 5):
    ERA5 = ds.where(ds.cluster == i, drop=True)
    
    ## open as geopandas dataframe
    df = ERA5.to_dataframe()
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat), crs="EPSG:4326")

    ### Code is based on https://james-brennan.github.io/posts/fast_gridding_geopandas/

    ### BUILD A GRID 
    # total area for the grid
    # xmin, ymin, xmax, ymax= gdf.total_bounds
    xmin, ymin, xmax, ymax= [-175., 20., -85.,  63.]
    # how many cells across and down
    n_cells=100
    cell_size = (xmax-xmin)/n_cells
    # projection of the grid
    crs = "EPSG:4326"
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            # bounds
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
    cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                     crs=crs)

    merged = gpd.sjoin(gdf, cell, how='left', predicate='within')

    # make a simple count variable that we can sum
    merged['n_traj']=1
    # Compute stats per grid cell -- aggregate fires to grid cells with dissolve
    dissolve = merged.dissolve(by="index_right", aggfunc="count")
    # put this into cell
    cell.loc[dissolve.index, 'n_traj'] = dissolve.n_traj.values
    print(cell['n_traj'].max())
    cell_lst.append(cell)

1086.0
2911.0
1834.0
2040.0


In [30]:
ext = [-130., -90., 25., 55.] 
fmt = 'png'

# Set up projection
datacrs = ccrs.PlateCarree()  ## the projection the data is in
mapcrs = ccrs.PlateCarree() ## the projection you want your map displayed in

# Set tick/grid locations
tx = 10
ty = 5
dx = np.arange(ext[0],ext[1]+tx,tx)
dy = np.arange(ext[2],ext[3]+ty,ty)

nrows = 1
ncols = 5

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1], width_ratios = [1, 1, 1, 1, 0.05], wspace=0.01, hspace=0.1)
## use gs[rows index, columns index] to access grids

fig = plt.figure(figsize=(30.0 ,7.))
fig.dpi = 600
fname = path_to_figs + 'heatmaps'

# Add color bar axis
cbax = plt.subplot(gs[:,-1]) # colorbar axis

row_idx = [0, 0, 0, 0]
col_idx = [0, 1, 2, 3]
llat_lst = [True, False, False, False]
HUC8_lbls = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']
for i, cell in enumerate(cell_lst):
    ax = fig.add_subplot(gs[row_idx[i],col_idx[i]], projection=mapcrs)
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy,left_lats=llat_lst[i], right_lats=False)
    

    ax.add_feature(cfeature.STATES, edgecolor='0.4', linewidth=0.8)
    ax.set_title(HUC8_lbls[i], loc='left', fontsize=14)

    ## plotting based off of https://geopandas.org/en/stable/docs/user_guide/mapping.html
    cf = cell.plot(ax=ax, column='n_traj', cmap=cmo.dense, vmin=0, vmax=800, edgecolor=None, legend=True, cax=cbax,
                  legend_kwds={"label": "Trajectory Frequency (#)", "orientation": "vertical"})

    ## add in four focus watersheds
    basin.plot(ax=ax, edgecolor='white', color='None', zorder=99)

    ax.set_extent(ext, datacrs)

# cb = Colorbar(ax = cbax, mappable = cf, orientation = 'vertical', ticklocation = 'right')
# cb.set_label('Trajectory Frequency (#)', fontsize=11)
# cb.ax.tick_params(labelsize=12)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi, transparent=True)
plt.show()
fig.clf()

In [ ]:
ext = [-130., -85., 25., 60.] 
fmt = 'png'

# Set up projection
datacrs = ccrs.PlateCarree()  ## the projection the data is in
mapcrs = ccrs.Mercator() ## the projection you want your map displayed in

# Set tick/grid locations
tx = 10
ty = 5
dx = np.arange(ext[0],ext[1]+tx,tx)
dy = np.arange(ext[2],ext[3]+ty,ty)

nrows = 1
ncols = 5

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1], width_ratios = [1, 1, 1, 1, 0.05], wspace=0.01, hspace=0.1)
## use gs[rows index, columns index] to access grids

fig = plt.figure(figsize=(30.0 ,7.))
fig.dpi = 600
fname = path_to_figs + 'cluster_trajectories_test'

# enumerate through seasons
HUC8_lbls = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']
row_idx = [0, 0, 0, 0]
col_idx = [0, 1, 2, 3]
llat_lst = [True, False, False, False]
    
for k in range(0, 3): ## loop through each cluster type
    ax = fig.add_subplot(gs[row_idx[k], col_idx[k]], projection=mapcrs)
    ax.set_title(HUC8_lbls[k], loc='left', fontsize=14)
    ax = draw_basemap(ax, extent=ext, xticks=dx, yticks=dy, left_lats=llat_lst[k], right_lats=False)
    ax.set_extent(ext, datacrs)
    ax.add_feature(cfeature.STATES, edgecolor='0.4', linewidth=0.8)
        
    for j, HUC8 in enumerate(HUC8_lst): # loop through each HUC8
        tmp = ds.sel(HUC8=HUC8) ## select each HUC8
        tmp = tmp.where(tmp.cluster == k+1, drop=True)

        # need this to fix annotate transform
        # transform = datacrs._as_mpl_transform(ax)
        nevents = len(ds.start_date)
        ## Add different points
        for i in range(nevents):
            data = ds.isel(start_date=i)
            y_lst = data.lat.values
            x_lst = data.lon.values
            z_lst = data.IVT.values
            # ax.plot(x_lst, y_lst, c='gray', transform=datacrs, alpha=0.2)
            cf = ax.scatter(x_lst, y_lst, c=z_lst, vmin=0, vmax=1000, cmap=cmo.thermal_r, marker='.', transform=datacrs, alpha=0.8, s=6)

# Add color bar
cbax = plt.subplot(gs[:,-1]) # colorbar axis
cb = Colorbar(ax = cbax, mappable = cf, orientation = 'vertical', ticklocation = 'right')
# cb.set_label('IVT (kg m$^{-1}$ s$^{-1}$)', fontsize=11)
cb.set_label('IVT (kg m$^{-1}$ s$^{-1}$)', fontsize=11)
cb.ax.tick_params(labelsize=12)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi, transparent=True)
plt.show()
fig.clf()


KeyboardInterrupt

